In [1]:
import roslibpy
import time

import opentnsim
import pandas as pd
import simpy.rt
import shapely.geometry


In [2]:
# Connect to the ros websocket bridge
client = roslibpy.Ros(host='localhost', port=9090)

def on_ready(*args, **kwargs):
    print('Connected', client.is_connected)
client.on_ready(on_ready)

In [ ]:
# Start a non-blocking event loop
# The client runs in a background thread.
client.run()

In [7]:
class HasRos:
    def __init__(self, client, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client = client

class RosListener(HasRos):
    def __init__(self, topic_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.topic_name = topic_name
        topic_type = self.client.get_topic_type(topic_name)
        # assert topic_type has latitude longitude
        self.topic = roslibpy.Topic(
            self.client, 
            topic_name, 
            topic_type
            
        )
    def subscribe(self):
        self.topic.subscribe(self.on_message)
    def on_message(self, message):
        self.last_message = message
        if isinstance(self, opentnsim.core.Locatable):
            self.geometry = shapely.geometry.Point(message['longitude'], message['latitude'])

class RosPublisher(opentnsim.core.Identifiable, HasRos):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        topic_name = f"/opentnsim_waypoint_{self.name}/log"
        topic_type = 'std_msgs/String'
        self.publish_topics['str'] = roslibpy.Topic(
            self.client, 
            topic_name, 
            topic_type
        )
        topic_name = f"/opentnsim_waypoint_{self.name}/pos"
        topic_type = 'sensor_msgs/NavSatFix'
        self.publish_topics['pos'] = roslibpy.Topic(
            self.client, 
            topic_name, 
            topic_type
        )
    def publish(self, message: str):
        self.publish_topic.publish(roslibpy.Message({'data': message}))
    
class WaypointPublisher(RosPublisher):
    def __init__(self, *args, **kwargs):
        super().__init__(*args ,**kwargs)
    def publish_next_waypoint(self):
        # LOGIC TO DETERMINE NEXT WAYPOINT HERE
        self.publish_topic.publish(roslibpy.Message({"data": }))

Ship = type(
    'Ship', 
    (
        opentnsim.core.SimpyObject, 
        opentnsim.core.Locatable,
        RosListener, 
        RosPublisher
    ), 
    {}
)

SyntaxError: invalid syntax (2592534274.py, line 50)

In [6]:

env = simpy.rt.RealtimeEnvironment()

ship = Ship(
    env=env, 
    geometry=shapely.geometry.Point(4.372, 52.0015), 
    client=client, 
    topic_name='/RAS_TN_GR/geopos_est', 
    name="ship"
)
env.run()

In [ ]:
for i in range(3):
    time.sleep(1)
    print(ship.geometry.wkt)